# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.8 Chat Completions模型的Function calling功能详解

- Function calling功能诞生背景

&emsp;&emsp;尽管大语言模型的知识储量巨大，且具备非常强大的涌现能力，但很多时候我们实际使用大语言模型时仍然会明显的感受到模型能力上的局限，例如模型无法获取最新的信息、模型只能给出文字的建议但无法直接帮我们解决某些问题（如自动回复邮件、自动查询订阅机票等）。毫无疑问，这些问题的存在极大程度限制了大语言模型的实际应用价值的进一步拓展，而在2023年4月，AutoGPT项目的横空出世则为这些解决问题提出了一个极具潜力的方案——那就是进一步赋予大语言模型调用外部工具API的能力，从而大幅拓展大语言模型的能力。例如如果我们能够让GPT模型调用谷歌搜索API（Google Custom Search JSON API），则模型就可以实时获取和用户问题相关的一系列搜索结果，并结合这些结果和原生的知识库来回答用户的问题，从而解决模型无法获取最新信息这一问题；再比如，如果我们能够让GPT模型调用谷歌邮箱API（Gmail API），则可以自动让GPT模型读取邮件，并自动进行回复等等。而根据AutoGPT不大的项目规模来看，让GPT模型调用外部工具API其实并不复杂。

> 什么是谷歌搜索API？类比下ChatGPT：我们可以在网页端和ChatGPT问答，我们也可以在代码环境调用ChatGPT背后的模型——gpt-3.5模型进行问答，而ChatCompletions.create就可以看成是ChatGPT的API，类似的，谷歌搜索也都有API，我们可以在代码环境中调用搜索引擎API来完成搜索。

&emsp;&emsp;而伴随着AutoGPT的爆火，人们在不断加深大语言模型的技术认知的同时，也逐渐发现了大语言模型落地应用的终极形态——即以大语言模型为“大脑”，打造由大语言模型驱动的AI应用。可以说，对于大部分领域，大语言模型的接入都将带来革命性的影响。围绕这一市场潜力巨大的AI应用开发需求，LangChain框架应运而生。可以说，LangChain框架是在AutoGPT基础上，首次系统梳理了一个围绕大语言模型的AI应用所需要具备的基本要素，而在LangChain设定的AI应用开发范式中，调用外部工具的API是则是整个AI应用开发流程中不可或缺的重要环节。基于此，LangChain在agent模块中提供了一整套通用的大语言模型调用外部工具API的工具，在很长一段时间，借助LangChain来让大语言模型具备调用外部工具API的能力，是广大开发者选择使用LangChain的最核心的原因之一。

- Function calling功能简介

&emsp;&emsp;在这一基本背景下，经过数月的研发和优化，OpenAI在0613的更新中为目前最先进的Chat类模型增加了Function calling功能，该功能的本质是让大语言模型调用外部函数的能力，即Chat模型可以不再仅仅根据自身的数据库知识进行回答，而是可以额外挂载一个函数库，然后根据用户提问进行函数库检索，根据实际需求调用外部函数并获取函数运行结果，再基于函数运行结果进行回答。其基本过程如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307061638380.png" alt="7055021e99cf56e7128c4348f864c5e" style="zoom:33%;" />

而这个外部挂载的函数库，可以是简单的自定义函数，也可以是一个封装了外部工具API的功能型函数（例如一个可以调用谷歌搜索的函数、或者一个可以获取天气信息的函数）。而在OpenAI的精妙设计下，Function calling功能的实现过程也并不复杂，在编写问答函数时，我们只需要在ChatCompletions.create函数中进行参数设置、并提前定义好外部函数库即可，而在Chat模型执行Function calling时，模型会根据用户提问的语义自动检索并挑选合适的函数进行使用，整个过程并不需要人工手动干预指定使用某个函数，大预言模型能够充分发挥自身的语义理解优势，在函数库中自动挑选合适函数进行运行，并给出问题的答案。

&emsp;&emsp;毫无疑问，有了外部函数库的功能加持，Chat模型的处理和解决问题的能力也必将再上一个台阶。同时，相比于此前必须借助LangChain的agent模块才能实现LLM和外部工具API的协同调用，现在Chat模型内部集成的Function calling功能实现过程更简单、开发门流程更加清晰、开发槛更低，而如此种种，也必将促进新一轮的以大语言模型为核心的AI应用的爆发。

&emsp;&emsp;接下来我们就将详细介绍Chat模型新增的Function calling的实现过程，并在之后的内容中给出一些非常高价值的Function calling实践案例。

In [1]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

import numpy as np
import pandas as pd

import json
import io

### 1.Chat外部函数库创建

&emsp;&emsp;尽管Chat模型的Function calling功能实现的思路非常清晰，但实际上，若要手动编写代码来实现这一功能，则会涉及外部函数库编写、ChatCompletion.create中functions参数和function_call参数的使用、用于获取函数结果的多轮对话流程编写等复杂过程。因此，我们先借助一个非常简单的外部函数调用示例来完整介绍Function calling功能实现的全部流程，然后再逐步进行高层函数的封装并尝试调用包含外部工具API的函数。

#### 1.1 外部函数库创建

&emsp;&emsp;即然要让Chat模型调用外部函数，首先需要准备的就是准备好这些“外部函数”。这里我们先自定义一个“特殊”的函数——陈明函数，然后尝试在和gpt对话中让模型调用并执行这个陈明函数。考虑到数据技术人非常核心的一个需求就是希望让大模型协助帮忙处理数据，因此我们定义的陈明函数就是一个专门用于数据处理的简单函数——即执行数据表的按行求和再减1。当然这个函数本身并没有任何特殊含义，只是用于测试Chat模型在合理提示下能否正常调用这个外部函数。

- 以字符串作为数据交换格式

&emsp;&emsp;当然，对于所有的需要和大模型通信的函数来说，函数输出的结构都必须是字符串类型才能够被大模型正常的识别。例如，我们可以通过.to_string()的方式，直接将Pandas对象类型转化为字符串，然后再让大模型进行识别：

In [2]:
# 创建一个DataFrame
df = pd.DataFrame({'x1':[1, 2], 'x2':[3, 4]})
df

,x1,x2
0,1,3
1,2,4


In [3]:
df_str = df.to_string()
df_str

'   x1  x2\n0   1   3\n1   2   4'

尽管可读性不强，但这个df_str对象实际上是一个包含了df全部信息的字符串对象。为了验证模型能否正确的识别df_str数据中信息，我们可以借助如下流程进行验证：

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "数据集df_str：'%s'" % df_str},
    {"role": "user", "content": "请帮我解释下df_str数据集"}
  ]
)

In [5]:
response

<OpenAIObject chat.completion id=chatcmpl-7cXnC0hwShqyh2r5JfWFqlYpf9O2w at 0x1eba40ea340> JSON: {
  "id": "chatcmpl-7cXnC0hwShqyh2r5JfWFqlYpf9O2w",
  "object": "chat.completion",
  "created": 1689420718,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "df_str \u6570\u636e\u96c6\u662f\u4e00\u4e2a\u7b80\u5355\u7684\u8868\u683c\u578b\u6570\u636e\u96c6\uff0c\u7531\u4e24\u5217\u6784\u6210\uff0c\u5206\u522b\u547d\u540d\u4e3a 'x1' \u548c 'x2'\u3002\u6570\u636e\u96c6\u4e2d\u6709\u4e24\u884c\u503c\u3002\u5728\u7b2c\u4e00\u884c\uff0cx1\u7684\u503c\u4e3a1\uff0cx2\u7684\u503c\u4e3a3\uff1b\u5728\u7b2c\u4e8c\u884c\uff0cx1\u7684\u503c\u4e3a2\uff0cx2\u7684\u503c\u4e3a4\u3002\u8fd9\u53ef\u80fd\u662f\u4e00\u4e9b\u6570\u503c\u6570\u636e\uff0c\u4f46\u662f\u4e0d\u77e5\u9053\u5177\u4f53\u4ee3\u8868\u4ec0\u4e48\u610f\u4e49\uff0c\u9700\u8981\u53c2\u8003\u5177\u4f53\u7684\u4e0a\u4e0b\u6587\u73af\u5883\u3002"
      },
      "fini

In [6]:
response.choices[0].message['content']

"df_str 数据集是一个简单的表格型数据集，由两列构成，分别命名为 'x1' 和 'x2'。数据集中有两行值。在第一行，x1的值为1，x2的值为3；在第二行，x1的值为2，x2的值为4。这可能是一些数值数据，但是不知道具体代表什么意义，需要参考具体的上下文环境。"

能够发现，模型能够很好的对字符串形式传入的数据集进行解读。

&emsp;&emsp;而有的时候，可能也需要围绕大模型输出的字符串形式的数据集进行转化，将其转化为Dataframe以便于后续处理。具体转化方法如下：

In [7]:
# 使用StringIO将字符串转换为文件对象
data = io.StringIO(df_str)

# 使用read_csv()函数读取数据，并设置第一列为索引
df_new = pd.read_csv(data, sep='\s+', index_col=0)

df_new

,x1,x2
0,1,3
1,2,4


该转化过程分为两步，其一是将df_str转化为文件对象，然后再将其读取为Datafrme类型。当然，这里的data对象本质上是一个类文件对象，从对象功能上来说，几乎可以等同于一个文件对象，只不过该对象只是通过磁盘映射构建的对象，并不会真正创建一个本地文件。截止目前，借助_io.StringIO对象进行字符串到Dataframe的转化也是最高效、最稳定的转化方法。

In [8]:
type(data)

_io.StringIO

- 以JSON格式作为数据交换格式

&emsp;&emsp;但很明显的是，字符串类型对象的可读性并不强，而且很多时候，大模型也并不能非常准确的将字符串对象识别为DataFrame对象类型。一个更为通用的方法是借助JSON格式进行跨函数和跨变成环境的通信，这也是OpenAI官方更加推荐的通信方式，同时，OpenAI的大语言模型也能够根据实际情况输出JSON格式的字符串。

&emsp;&emsp;所谓JSON格式对象，其本质是一种轻量级的数据交换格式，它基于ECMAScript的一个子集，采用完全独立于编程语言的文本格式来存储和表示数据，在python中的表现其实仍然是字符串类型，相当于是一种能够表示原始数据格式的字符串。例如对于df来说，我们可以通过pandas中的to_json()方法将其转化为json对象类型：

In [9]:
df

,x1,x2
0,1,3
1,2,4


In [10]:
df.to_json(orient='records')

'[{"x1":1,"x2":3},{"x1":2,"x2":4}]'

能够发现，转化结果仍然是通过字符串类型进行表示，但实际上字符串的内容是一个object类型的JSON对象，即为一个无序的集合。也就是说，在JSON中，可以使用object对象来表示Python中DataFrame，同时orient='records'参数表示每一行（记录）被转换为一个JSON对象，所有的这些对象被放在一个JSON数组（list）中。

> 这种不同对象类型之间的转化过程，更像是翻译的过程，即把一个DataFrame对象翻译成JSON对象。

并且，我们也可以简单验证Chat模型是否能够识别JSON对象：

In [11]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "system", "content": "数据集df_json：'%s'" % df.to_json(orient='records')},
    {"role": "user", "content": "请帮我解释下df_json数据集"}
  ]
)
response.choices[0].message['content']

'df_json 数据集是一个包含两个JSON对象的字符串数据集。每个JSON对象由两个属性组成：x1和x2。\n- 第一个JSON对象具有属性x1的值为1，属性x2的值为3。\n- 第二个JSON对象具有属性x1的值为2，属性x2的值为4。\n\n在Python或类似的数据处理语言中，此数据通常会被读取并转化为一个数据结构，如列表或数据框（data frame），以便于数据处理和分析。'

能够发现，模型能够正常识别JSON对象类型。并且，相比.to_string()翻译而成的字符串对象，JSON对象本身的可读性更强：

In [12]:
df.to_string()

'   x1  x2\n0   1   3\n1   2   4'

并且，我们还可以通过pd.read_json函数，将一个JSON的Object对象“翻译”成Pandas对象：

In [13]:
pd.read_json(df.to_json(orient='records'))

,x1,x2
0,1,3
1,2,4


不难发现，其实转化为字符串或者JSON格式，都可以完成DataFrame对象类型的数据交换。本节我们先采用更为简单的字符串形式作为DataFrame对象的数据交换格式，在Ch.10的案例中，我们会重点介绍另一种方法，即采用JSON格式作为数据交换格式。

- 陈明函数与函数库创建

&emsp;&emsp;接下来我们创建陈明函数，该函数实际计算过程并不复杂，无非就是将数据集对象按行求和再减1，计算过程如下：

In [10]:
np.sum(df_new, axis=1) - 1

0    3
1    5
dtype: int64

但是，作为支持Function calling功能的外部函数，考虑到要和Chat大模型进行通信，陈明函数的输入由大模型提供（而大模型的输出对象都是字符串），因此函数要求输入对象必须是字符串对象；此外，OpenAI也明确规定支持Function calling功能的外部函数给大模型返回的结果类型必须是json字符串类型，因此陈明函数的输出结果需要先将Dataframe对象转化为字符串对象，然后再将字符串对象转化为json字符串类型对象。当然，为了增加代码的可读性，也需要注明函数说明及参数和输出结果说明。具体陈明函数定义过程如下：

In [44]:
def chen_ming_algorithm(data):
    """
    陈明算法函数，该函数定义了一种特殊的数据集计算过程
    :param data: 必要参数，表示带入计算的数据表，用字符串进行表示
    :return：陈明函数计算后的结果，返回结果为表示为JSON格式的Dataframe类型对象
    """
    data = io.StringIO(data)
    df_new = pd.read_csv(data, sep='\s+', index_col=0)
    res = np.sum(df_new, axis=1) - 1
    return json.dumps(res.to_string())

In [8]:
df

,x1,x2
0,1,3
1,2,4


In [9]:
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [10]:
chen_ming_algorithm(df_str)

'"0    3\\n1    5"'

> 这里需要注意函数说明和参数解释的书写风格，一般来说不同的程序员会有不同的函数备注书写风格，课程中采用的函数备注风格是sklearn函数的备注风格，该备注方法会采用三个引号标注接下来是函数说明，并在第一行文字开始描述函数功能，而在后续的参数解释过程中，:param表示对某参数的解释，参数解释的时候需要说明参数性质（必要参数还是可选参数，可选参数的话需要注明可选参数的默认值），然后需要说明参数对象类型，并进行参数功能解释。在备注的最后，:return表示函数返回结果，需要解释返回结果内容和返回的对象类型。

> 在本节内容介绍的Chat外链函数库函数的编写过程中，函数备注非常重要，后续有一个手动编写的自动函数功能识别函数，需要读取函数备注并让大模型对函数功能及参数格式进行JSON格式翻译。

&emsp;&emsp;在Chat模型实际执行Function calling功能时，是从一个函数库中筛选合适的函数进行调用，因此我们还需要准备一个函数库。当然最简单的情况下，函数库允许只包含一个函数：

In [15]:
available_functions = {
            "chen_ming_algorithm": chen_ming_algorithm,
        }

In [45]:
chen_ming_algorithm.__name__

'chen_ming_algorithm'

函数库对象也必须是一个字典，一个键值对代表一个函数，其中Key是代表函数名称的字符串，而value表示对应的函数。

#### 1.2 functions参数解释与定义过程

&emsp;&emsp;在准备好外部函数及函数库之后，接下来非常重要的一步就是需要将外部函数的信息以某种形式传输给Chat模型。此时就需要使用到ChatCompletion.create函数的functions参数，类似于messages参数是用于向模型传输消息，functions参数专门用于向模型传递当前可以调用的外部函数信息。并且，从参数的具体形式来看，functions参数和messages参数也是非常类似的——都是包含多个字典的list。对于messages来说，每个字典都是一条信息，而对于functions参数来说，每个字典都是一个函数。在大预言模型实际进行问答时，会根据functions参数提供的信息对各函数进行检索。

&emsp;&emsp;很明显，functions参数对于Chat模型的Function calling功能的实现至关重要。接下来我们详细解释functions中每个用于描述函数的字典编写方法。总的来说，每个字典都有三个参数（三组键值对），各参数（Key）名称及解释如下：

- name：代表函数函数名称字的符串，必选参数，按照要求函数名称必须是 a-z、A-Z、0-9，或包含下划线和破折号，最大长度为 64。需要注意的是，name必须输入函数名称，而后续模型将根据函数名称在外部函数库中进行函数筛选；
- description：用于描述函数功能的字符串，虽然是可选参数，但该参数传递的信息实际上是Chat模型对函数功能识别的核心依据。即Chat函数实际上是通过每个函数的description来判断当前函数的实际功能的，若要实现多个备选函数的智能挑选，则需要严谨详细的描述函数功能；（需要注意的是，在某些情况下，我们会通过其他函数标注本次对话特指的函数，此时模型就不会执行这个根据描述信息进行函数挑选的过程，此时是可以不设置description的。）
- parameters：函数参数，必选参数，要求遵照JSON Schema格式进行输入，JSON Schema是一种特殊的JSON对象，专门用于验证JSON数据格式是否满足要求。

&emsp;&emsp;例如，对于陈明函数，我们需要创建如下字典来对其进行完整描述：

In [11]:
chen_ming_function = {"name": "chen_ming_algorithm",
                      "description": "用于执行陈明算法的函数，定义了一种特殊的数据集计算过程",
                      "parameters": {"type": "object",
                                     "properties": {"data": {"type": "string",
                                                             "description": "执行陈明算法的数据集"},
                                                   },
                                     "required": ["data"],
                                    },
                     }

其中，name就是函数chen_ming_algorithm名称的字符串表示形式：

In [82]:
chen_ming_algorithm.__name__

'chen_ming_algorithm'

当然，字典中的前两个键值对并不难理解，比较关键、同时也是编写起来较为复杂的是"parameters"参数。

---

**<center>JSON与JSON Schema对象解释**

&emsp;&emsp;首先，"parameters"参数是一个按照JSON Schema格式编写的字典，总的来说，这个字典中包含了函数参数这个对象的全部信息，同时其本身的编写格式也可以用于验证后续传入这个函数的参数是否满足格式要求。这里我们需要重点介绍下什么是JSON Schema对象，从形式上来说，以下字典其实就是一个JSON Schema对象：

In [15]:
{"type": "object",
 "properties": {"data": {"type": "string",
                         "description": "执行陈明算法的数据集"}
               },
 "required": ["data"]
}

{'type': 'object',
 'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
 'required': ['data']}

能够发现，这个字典其实是由多个字典组成，其中type、properties、required等都是字典中的关键字，从语法结构上来说，（和JSON对象类似）满足JSON Schema语法要求的字典对象，都可以看成是JSON Schema对象。而对于JSON Schema对象来说，字典中的关键字都是有独特含义的，要理解这些关键含义，就必须理解JSON Schema对象的作用。

&emsp;&emsp;从功能上来说，JSON Schema对象的根本作用是用于用于描述某一类JSON对象的格式要求。我们知道，JSON（JavaScript Object Notation）是一种轻量级的数据交换格式，它基于JavaScript的一个子集。JSON采用完全独立于语言的文本格式，这使得JSON成为理想的数据交换语言。尽管JSON是JavaScript的一个子集，但JSON是独立于语言的。这意味着无论使用什么编程语言，都可以解析JSON数据，并且可以将JSON数据转换为本地对象。例如，我们可以将df_new转化为JSON对象：

In [17]:
df_new

,x1,x2
0,1,3
1,2,4


In [18]:
json_str = df_new.to_json(orient='records')
json_str

'[{"x1":1,"x2":3},{"x1":2,"x2":4}]'

在这个例子中，to_json()函数将DataFrame转换为了一个JSON格式的字符串。orient='records'参数表示每一行（记录）被转换为一个JSON对象，所有的这些对象被放在一个JSON数组（list）中。

&emsp;&emsp;而不同于JSON对象用于保存实际的对象信息，JSON Schema对象则专门用于描述某JSON对象的数据格式要求，例如在这个例子中，json_str是一个包含多个对象的数组，每个对象都有"x1"和"x2"两个属性，且它们的值都是数字，此时我们就可以用如下JSON Schema对象来描述json_str对象的数据格式要求：

In [31]:
json_Schema_str = {
                   "type": "array",
                   "items": {
                     "type": "object",
                     "properties": {
                       "x1": {
                         "type": "number"
                       },
                       "x2": {
                         "type": "number"
                       }
                     },
                     "required": ["x1", "x2"]
                   }
                 }
json_Schema_str

{'type': 'array',
 'items': {'type': 'object',
  'properties': {'x1': {'type': 'number'}, 'x2': {'type': 'number'}},
  'required': ['x1', 'x2']}}

在这个JSON Schema中：

- "type": 本身指代 JSON 对象数据的类型，如"string"、"number"、"object"、"array"、"boolean" 或 "null"，很明显，这里的JSON对象是个"array"，表示这个Schema描述的是一个数组。
- "items"：当 JSON 数据的类型为 "array" 时，定义其元素的结构。在这个例子中，每个元素都是一个对象（object），在JSON中，一个对象（object）指的是由零个或多个键值对组成的无序集合。
- 在"items"对象中，"type": "object"表示每个元素都是一个对象，"properties"关键字描述了对象的属性，"x1"和"x2"都是对象的属性，它们的类型都是"number"。
- "required"：用于定义哪些属性是必需的。关键字指定了每个对象都必须有"x1"和"x2"这两个属性。       


这个JSON Schema可以用来验证类似json_str这样的JSON对象。如果一个JSON对象满足这个Schema，那么它就是一个包含多个对象的数组，每个对象都有"x1"和"x2"两个数字属性。

&emsp;&emsp;也就是说，JSON Schema本身并不包含某对象的具体内容信息，只包含某类对象的格式信息，并且我们也简单了解了JSON Schema对象一些关键词的基本解释。那么反观chen_ming_function['parameters']这个JSON Schema对象，应该就能明白其含义了：

In [23]:
chen_ming_function['parameters']

{'type': 'object',
 'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
 'required': ['data']}

In [ ]:
{'data':data}

首先，要求'type': 'object'，说明要求对应的JSON对象类型为object，即由零个或者多个键值对组成无序集合，其次'properties'表示当 JSON 数据的类型为 "object" 时，其属性的结构需要按照如下方式进行进行组织，换而言之就是这个无序集合必须包含哪些key，以及对应的value格式需要满足哪些要求。而{'data': {'type': 'string', 'description': '执行陈明算法的数据集'}}则说明这个无序集合里面必须包含一个名为data的key，而这个key对应的value，则需要满足如下要求：{'type': 'string', 'description': '执行陈明算法的数据集'}，即value必须是字符串，而description则用于说明这个value的实际作用，即这个value为“执行陈明算法的带入的数据集”。而chen_ming_function['parameters']的第三个关键词'required'，则用于说明这个JSON对象（也就是某无序集合）必须包含的元素列表，而'required': ['data']则要求无序集合必须包含一个key为data的键值对。

> 这里需要注意，若某函数没有任何必选参数，则用于描述该函数参数格式的JSON Schema对象的关键词'required'，可以填写为空字典，即"properties": {}。

&emsp;&emsp;综上，chen_ming_function['parameters']其实非常明确的定义了某类JSON对象类型。而我们为何需要编写chen_ming_function['parameters']，其实也是因为在Chat模型在进行对话时，传入和传出数据信息都是以类似JSON数据格式进行传输的：

In [27]:
response = openai.ChatCompletion.create(
  model="gpt-4-0613",
  messages=[
    {"role": "user", "content": "请帮我创建一个两行两列的数据集，两列的列名称分别是x1和x2，第一行数据为1、3，第二行为2、4"}
  ]
)
response.choices[0].message['content']

'在Python的pandas库中，你可以创建一个DataFrame来实现这个要求。\n\n在下面的代码中，我们首先导入pandas库，然后使用 `pd.DataFrame` 函数来创建一个新的DataFrame。`columns` 参数用于指定列名称，而 `data` 参数则用于输入具体的数据。\n\n```python\nimport pandas as pd\n\ndata = pd.DataFrame(columns=["x1", "x2"], data=[[1, 3], [2, 4]])\nprint(data)\n```\n\n当你运行这段代码时，你将得到以下输出：\n\n```\n   x1  x2\n0   1   3\n1   2   4\n```'

In [30]:
response

<OpenAIObject chat.completion id=chatcmpl-7bPFX8ntFs8gLnaAbiu3bQS7S9x9g at 0x266dccc3810> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\u5728Python\u7684pandas\u5e93\u4e2d\uff0c\u4f60\u53ef\u4ee5\u521b\u5efa\u4e00\u4e2aDataFrame\u6765\u5b9e\u73b0\u8fd9\u4e2a\u8981\u6c42\u3002\n\n\u5728\u4e0b\u9762\u7684\u4ee3\u7801\u4e2d\uff0c\u6211\u4eec\u9996\u5148\u5bfc\u5165pandas\u5e93\uff0c\u7136\u540e\u4f7f\u7528 `pd.DataFrame` \u51fd\u6570\u6765\u521b\u5efa\u4e00\u4e2a\u65b0\u7684DataFrame\u3002`columns` \u53c2\u6570\u7528\u4e8e\u6307\u5b9a\u5217\u540d\u79f0\uff0c\u800c `data` \u53c2\u6570\u5219\u7528\u4e8e\u8f93\u5165\u5177\u4f53\u7684\u6570\u636e\u3002\n\n```python\nimport pandas as pd\n\ndata = pd.DataFrame(columns=[\"x1\", \"x2\"], data=[[1, 3], [2, 4]])\nprint(data)\n```\n\n\u5f53\u4f60\u8fd0\u884c\u8fd9\u6bb5\u4ee3\u7801\u65f6\uff0c\u4f60\u5c06\u5f97\u5230\u4ee5\u4e0b\u8f93\u51fa\uff1a\n\n```\n   x1  x2\n0   1   3\n1 

而Chat模型在调用Function calling功能时，函数所需要的参数是通过对话信息进行传输的，该参数在执行过程中会被转化为JSON数据然后输入函数，此时很明显需要有一个验证函数参数对象是否满足格式要求的过程，因此，借助JSON Schema对象进行参数结构验证，就是非常好的选择。而就chen_ming_function['parameters']这个JSON Schema对象来说，申明了参数需要给data参数传入一个'string'（一个已经转化为'string'的数据集），并且data这个参数也是必选参数。

> 这里需要注意的是，大语言模型的数据传输除了借助字符串类型对象进行执行外，我们还可以通过JSON格式进行数据传输，该方法稍后会进行介绍。

> functions的官方参数说明是：A list of functions the model may generate JSON inputs for.也侧面说明模型是通过JSON格式进行数据传输。

> 更多信息，详见JSON Schema编写规范官方说明：https://json-schema.org/understanding-json-schema/

---

&emsp;&emsp;在按照格式编写了陈明函数的函数信息之后，我们需要将chen_ming_function添加到functions对象中，functions是一个包含了多个字典信息的列表，后续我们也是带入functions到模型中进行运行，而不是chen_ming_function。

> 类似于输入模型的messages参数是包含多个message的列表，输入模型的functions也是包含多个函数描述的列表，只不过此时我们只有一个函数，因此functions列表中只有一个字典。

In [12]:
chen_ming_function

{'name': 'chen_ming_algorithm',
 'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
 'parameters': {'type': 'object',
  'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
  'required': ['data']}}

In [13]:
functions = [chen_ming_function]
functions

[{'name': 'chen_ming_algorithm',
  'description': '用于执行陈明算法的函数，定义了一种特殊的数据集计算过程',
  'parameters': {'type': 'object',
   'properties': {'data': {'type': 'string', 'description': '执行陈明算法的数据集'}},
   'required': ['data']}}]

### 2.Function calling功能实现

#### 2.1 First response

&emsp;&emsp;在进行了一系列基础准备工作之后，接下来我们尝试在Chat模型对话执行Function calling功能。这里我们创建如下messages：

In [19]:
df_str

'   x1  x2\n0   1   3\n1   2   4'

In [20]:
"数据集data：%s" % df_str

'数据集data：   x1  x2\n0   1   3\n1   2   4'

In [21]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请在数据集data上执行陈明算法"}
]

在这个messages中，我们通过system message申明了data数据，然后通过user message向模型发送请求，让模型在data数据集上执行陈明算法。首先我们测试如果只输入这个信息而不输入外部函数库的时候，模型能否知道陈明函数并进行计算：

In [22]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages 
    )
response_message = response["choices"][0]["message"]

In [25]:
response

<OpenAIObject chat.completion id=chatcmpl-7bn4VbUKStTdJB0jG5Nu4hCuhun2f at 0x1fa02c81ae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\u5bf9\u4e0d\u8d77\uff0c\u4f60\u53ef\u80fd\u88ab\u8bef\u5bfc\u4e86\u3002\u76ee\u524d\u6ca1\u6709\u53eb\u505a\"\u9648\u660e\u7b97\u6cd5\"\u7684\u516c\u8ba4\u7684\u8ba1\u7b97\u673a\u7b97\u6cd5\u6216\u7edf\u8ba1\u5b66\u65b9\u6cd5\u3002\u60a8\u80fd\u5426\u63d0\u4f9b\u66f4\u591a\u76f8\u5173\u4fe1\u606f\u6216\u8005\u660e\u786e\u60f3\u8981\u5b8c\u6210\u7684\u64cd\u4f5c\uff1f \u5982\u679c\u4f60\u5728\u5bfb\u6c42\u7279\u5b9a\u5206\u6790\u6216\u5904\u7406\u8fd9\u4e2a\u6570\u636e\u96c6\u7684\u65b9\u6cd5\uff0c\u6211\u4f1a\u975e\u5e38\u4e50\u610f\u5e2e\u52a9\u3002",
        "role": "assistant"
      }
    }
  ],
  "created": 1689241123,
  "id": "chatcmpl-7bn4VbUKStTdJB0jG5Nu4hCuhun2f",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 92,
    "prompt_t

In [23]:
response_message["content"]

'对不起，你可能被误导了。目前没有叫做"陈明算法"的公认的计算机算法或统计学方法。您能否提供更多相关信息或者明确想要完成的操作？ 如果你在寻求特定分析或处理这个数据集的方法，我会非常乐意帮助。'

很明显，这个陈明函数过于特殊，并不包含在gpt-4模型的知识库中，因此模型无法进行回答。

&emsp;&emsp;接下来我们尝试将函数库相关信息输入给Chat模型，这里需要额外设置两个参数，其一是functions参数，用于申明外部函数库当前情况，其二则是需要设置function_call参数，该参数用于控制是否执行Function calling功能，该参数有三种不同的取值，默认取值为none，表示不需要调用外部函数，不执行Function calling功能，此时functions参数不需要进行额外设置。而如果设置了functions参数，则function_call参数会默认修改为'auto'（当然也可以手动填写该参数），表示模型将根据用户实际对话情况，有选择性的自动挑选合适函数进行执行，而若想让模型在本次对话中特定执行functions中的某个函数，则可以通过输入如下形式的字典：{"name":\ "my_function"}进行申明，此时模型不再会自动挑选模型，而是会在functions中挑选"my_function"进行执行。

&emsp;&emsp;这里我们尝试让模型自动挑选函数来进行执行，参数设置方法与执行过程如下：

In [141]:
response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )

观察此时response结果：

In [142]:
response

<OpenAIObject chat.completion id=chatcmpl-7bQI0Lr6Nns4xoOUW4JyPXBjcWB36 at 0x266deb93b30> JSON: {
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}",
          "name": "chen_ming_algorithm"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1689153548,
  "id": "chatcmpl-7bQI0Lr6Nns4xoOUW4JyPXBjcWB36",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 130,
    "total_tokens": 166
  }
}

In [143]:
response_message = response["choices"][0]["message"]
response_message

<OpenAIObject at 0x266deb9d0e0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}",
    "name": "chen_ming_algorithm"
  },
  "role": "assistant"
}

能够发现，此时返回的message中content为空，而增了一个"function_call"字典，该字典包含两个键值对，观察Key名称不难发现，Key为"name"的键值对表示完成该对话需要调用的函数名称，也就是chen_ming_algorithm，而Key为"arguments"的键值对则表示需要传入该函数的参数。注意，这条消息仍然是assistant传来的消息，只不过这条消息不包含content，而包含function_call关键词，表示这条assistant消息需要调用外部函数。而function_call关键词也是messages参数可以包含的备选关键词之一。

&emsp;&emsp;接下来我们将这两个核心信息分别保存为function_name和function_args对象：

In [144]:
# 完成对话需要调用的函数名称
function_name = response_message["function_call"]["name"]
function_name

'chen_ming_algorithm'

In [145]:
# 具体的函数对象
fuction_to_call = available_functions[function_name]
fuction_to_call

<function __main__.chen_ming_algorithm(data)>

In [146]:
# 执行该函数所需要的参数
function_args = json.loads(response_message["function_call"]["arguments"])
function_args

{'data': 'x1  x2\n0   1   3\n1   2   4'}

需要注意的是，外部函数的计算过程仍然是在本地执行，即Chat模型并不会将代码读取到服务器上再进行在线计算，因此接下来我们需要根据模型返回的函数和函数参数，在本地完成函数计算，然后再将计算过程和结果保存为message并追加到messages后面，并第二次调用Chat模型分析函数的计算结果，并最终根据函数计算结果输出用户问题的答案。

#### 2.2 Second response

&emsp;&emsp;这里我们只需要借助\*\*方法，直接将function_args对象传入fuction_to_call中，即可一次性传输全部参数，\*\*方法的功能可以参考如下示例：

In [84]:
def function_to_call_test(a, b, c):
    return a + b + c

function_args_test = {'a': 1, 'b': 2, 'c': 3}

result = function_to_call_test(**function_args_test)

print(result)

6


\*\*方法其实是一种较为特殊、但同时也非常便捷的参数传递方法吗，该方法会将字典中的每个key对应的value传输到同名参数位中。接下来我们将function_args对象传入fuction_to_call中并完成计算：

In [147]:
function_response = fuction_to_call(**function_args)

In [148]:
function_response

'"0    3\\n1    5"'

能够发现，模型已经顺利完成计算。接下来我们在messages对象中追加两条消息，第一条消息是第一次模型返回的结果（即调用模型的assistant message），第二条消息则是外部函数计算结果，该条消息的role为function，且name为函数名称。这也是我们首次接触function message，和user、system、assistant message不同，function message必须要输入关键词name，且function message的内容源于外部函数执行的计算结果，并且需要手动进行输入。具体添加过程如下：

In [150]:
messages

[{'role': 'system',
  'content': '数据集data：   x1  x2\n0   1   3\n1   2   4，数据集以字符串形式呈现'},
 {'role': 'user', 'content': '请在数据集data上执行陈明算法'}]

In [151]:
# 追加第一次模型返回结果消息
messages.append(response_message)  

In [152]:
messages

[{'role': 'system',
  'content': '数据集data：   x1  x2\n0   1   3\n1   2   4，数据集以字符串形式呈现'},
 {'role': 'user', 'content': '请在数据集data上执行陈明算法'},
 <OpenAIObject at 0x266deb9d0e0> JSON: {
   "content": null,
   "function_call": {
     "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}",
     "name": "chen_ming_algorithm"
   },
   "role": "assistant"
 }]

In [153]:
# 追加function返回消息
messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

In [154]:
messages

[{'role': 'system',
  'content': '数据集data：   x1  x2\n0   1   3\n1   2   4，数据集以字符串形式呈现'},
 {'role': 'user', 'content': '请在数据集data上执行陈明算法'},
 <OpenAIObject at 0x266deb9d0e0> JSON: {
   "content": null,
   "function_call": {
     "arguments": "{\n\"data\": \"x1  x2\\n0   1   3\\n1   2   4\"\n}",
     "name": "chen_ming_algorithm"
   },
   "role": "assistant"
 },
 {'role': 'function',
  'name': 'chen_ming_algorithm',
  'content': '"0    3\\n1    5"'}]

接下来，再次调用Chat模型来围绕messages进行回答。需要注意的是，此时我们不再需要向模型重复提问，只需要简单的将我们已经准备好的messages传入Chat模型即可：

In [155]:
second_response = openai.ChatCompletion.create(
            model="gpt-4-0613",
            messages=messages,)

In [156]:
second_response["choices"][0]["message"]["content"]

'执行陈明算法后，得到的数据集为：\n\n   x1  x2\n0   3\n1   5'

能够发现，模型最终做出了准确回答。我们将答案中的字符串再次转化为Dataframe，结果如下：

In [160]:
df_str = '\n\n   x1  x2\n0   3\n1   5'

In [161]:
# 使用StringIO将字符串转换为文件对象
data = io.StringIO(df_str)

# 使用read_csv()函数读取数据，并设置第一列为索引
df_new = pd.read_csv(data, sep='\s+', index_col=0)

In [162]:
df_new

,x2
x1,
0,3
1,5


最终，经过两次Chat模型的调用以及一次本地函数的计算，Chat模型很好的完成了最初问题的回答。至此，我们也完整的实现了一次Chat模型的Function calling功能。为了方便Function calling功能对比，接下来我们测试如果不询问陈明宣发相关问题，模型是否还会调用模型：

In [33]:
messages=[
    {"role": "system", "content": "数据集data：%s，数据集以字符串形式呈现" % df_str},
    {"role": "user", "content": "请帮我介绍下data数据集"}
]

response = openai.ChatCompletion.create(
        model="gpt-4-0613",
        messages=messages,
        functions=functions,
        function_call="auto",  
    )
response_message = response["choices"][0]["message"]

In [34]:
response_message

<OpenAIObject at 0x1fa02cc3ae0> JSON: {
  "content": "\u8fd9\u4e2a\u6570\u636e\u96c6\u662f\u4e00\u4e2a\u7b80\u5355\u7684\u4e8c\u7ef4\u6570\u636e\u96c6\uff0c\u7531\u4e24\u4e2a\u53d8\u91cfx1\u548cx2\u7ec4\u6210\u3002\u5b83\u6709\u4e24\u884c\u8bb0\u5f55\uff0c\u6bcf\u884c\u4ee3\u8868\u4e00\u4e2a\u6570\u636e\u70b9\uff0c\u5177\u4f53\u5982\u4e0b\uff1a\n\n- \u6570\u636e\u70b9 1: x1 = 1, x2 = 3\n- \u6570\u636e\u70b9 2: x1 = 2, x2 = 4\n\n\u8fd9\u79cd\u5f62\u5f0f\u7684\u6570\u636e\u96c6\u901a\u5e38\u5728\u5404\u79cd\u6570\u636e\u79d1\u5b66\u548c\u673a\u5668\u5b66\u4e60\u6a21\u578b\u4e2d\u4f7f\u7528\u3002\u6bcf\u4e00\u884c\u8868\u793a\u4e00\u4e2a\u89c2\u5bdf\u7ed3\u679c\uff0c\u6bcf\u4e00\u5217\u4ee3\u8868\u4e00\u4e2a\u533a\u522b\u7684\u53d8\u91cf\u3002",
  "role": "assistant"
}

In [35]:
response_message['content']

'这个数据集是一个简单的二维数据集，由两个变量x1和x2组成。它有两行记录，每行代表一个数据点，具体如下：\n\n- 数据点 1: x1 = 1, x2 = 3\n- 数据点 2: x1 = 2, x2 = 4\n\n这种形式的数据集通常在各种数据科学和机器学习模型中使用。每一行表示一个观察结果，每一列代表一个区别的变量。'

能够发现，此时Chat模型不再会调用函数，返回的结果也只是一个简单的assistant message。这也说明Chat模型确实能够正确的识别语义，并根据语义有选择性的进行外部函数调用。

> 能够理解语义并做出灵活判断是大语言模型最核心的能力之一，而作为开发者，在很多开发环境中，若能很好的利用大预言模型的这个特性，则可以在很多环节极大程度提高开发效率。例如，如果我们觉得编写JSON Schema来描述函数参数非常复杂，或许我们也能利用大语言模型的语义理解能力，让模型帮我们编写。相关方法我们会在后续内容中进行介绍。

&emsp;&emsp;当然，就整个流程而言，不难发现Chat模型的Function calling功能实现逻辑还是非常清晰的，只是对于初学者而言，代码编写有一定的难度，且代码执行逻辑略微有些复杂，相比上一小节简单调用Chat模型进行原始知识库问答，Function calling过程涉及的参数更多，而且很多参数的参数名重复度较高，例如messages参数中的function_call关键字和ChatCompletion.create中的function_call参数等，极易混淆。接下来，我们先围绕上述过程进行一个完整的流程总结，然后再进行一些更高层次的函数封装，方便更加便捷的实现Function calling功能。

#### 2.3 Function calling功能实现流程总结

&emsp;&emsp;总的来说，上述Chat模型的Function calling实现流程可以总结为如下形式：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202307131805614.png" alt="Function calling流程总结 (2)" style="zoom:33%;" />